In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [65]:
PATTERN_HEADLINE_TEXT = re.compile(r"(?<=\"headline\">)[A-Öa-ö 0-9.!?,-s]+(?=<\/h1>)")
PATTERN_BROAD_TEXT = re.compile(r"(?<=fomaker.se\/idf\/1.0\">)[A-Öa-ö -0-9.!?,–]+(?=<\/element>)")
PATTERN_LINK_TEXT = re.compile(r"(?<=\"c-article__body__content\">)[\nA-Öa-ö -0-9.!?,–<>=:0-9]+")
ARTICLE_DATE = re.compile(r'(?<=datetime\=)[A-Öa-ö 0-9.!?,-:"]+(?=\sitemprop\=\"datePublished\">)')

In [34]:
url = "https://gp.se/nyheter/världen/navalnyj-tvingas-titta-på-tv-i-fängelset-1.53513690"
response = requests.get(url)
soup = BeautifulSoup(response.content)

In [66]:
title = re.findall(PATTERN_LINK_TEXT, str(soup))

In [67]:
title

['\n<p>Aleksej Navalnyj avtjänar ett tre och ett halvt år långt fängelsestraff utanför huvudstaden Moskva, dömd för att ha brutit mot anmälningsplikten för sin villkorliga dom under tiden som han vårdades på sjukhus i Tyskland efter mordförsöket på honom.</p>\n<p>I en intervju med <a href="https://www.nytimes.com/2021/08/25/world/europe/navalny-jail-prison.html?" id="dvOsl4TxXrWBJGZ7">The New York Times</a> som är den första intervju som Navalnyj ger till en nyhetsredaktion sedan gripandet – berättar han om tiden i fängelset.</p>\n<h2>Filmer om kriget</h2>\n<p>Navalnyj berättar bland annat att han och andra fångar tvingas titta på statlig tv och propagandafilmer mer än åtta timmar om dagen, och att vakterna skriker åt dem som somnar.</p>\n<p>– Vi tittar på filmer om andra världskriget, eller om att våra atleter besegrade amerikaner eller kanadensare en dag för 40 år sedan, säger Navalnyj.</p>\n<p>Oppositionsledaren säger att tv-tittandet delas upp i fem pass om dagen, det första inleds

In [61]:
str(soup)

'<!DOCTYPE html>\n<html lang="sv">\n<head>\n<meta charset="utf-8"/>\n<meta content="ie=edge" http-equiv="X-UA-Compatible"/>\n<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>\n<script async="" src="https://x.klarnacdn.net/kp/lib/v1/api.js"></script>\n<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.4.1/jquery.js"></script>\n<script>\n            window.headerScripts = window.headerScripts || {}\n            window.headerScripts.cacheBuster = function() {\n                let cacheBuster = "";\n                if (window.location.href.indexOf("1.") !== -1) {\n                    cacheBuster = "?login=true";\n                    if (window.location.href.indexOf("?") !== -1) {\n                        cacheBuster = "&login=true";\n                    }\n                }\n\n                return cacheBuster;\n            }\n        </script>\n<script>\n            window.slm = window.slm || {};\n            window.slm

In [60]:
title

['</div>']

In [51]:
soup

<!DOCTYPE html>
<html lang="sv">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<script async="" src="https://x.klarnacdn.net/kp/lib/v1/api.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.4.1/jquery.js"></script>
<script>
            window.headerScripts = window.headerScripts || {}
            window.headerScripts.cacheBuster = function() {
                let cacheBuster = "";
                if (window.location.href.indexOf("1.") !== -1) {
                    cacheBuster = "?login=true";
                    if (window.location.href.indexOf("?") !== -1) {
                        cacheBuster = "&login=true";
                    }
                }

                return cacheBuster;
            }
        </script>
<script>
            window.slm = window.slm || {};
            window.slm.premiumLinks = window.sl

In [70]:
def collect_data(source_page):
    title = re.findall(PATTERN_HEADLINE_TEXT, source_page)
    date = re.findall(ARTICLE_DATE, source_page)
    collected = []
    for match in re.finditer(PATTERN_BROAD_TEXT, source_page):
        text = match.group() + " "
        span = match.span()
        data = (span, text)
        collected.append(data)

    if not collected:
        #return ["NULL"], "NULL", "NULL"
        return "NULL","NULL","NULL"
    if not title:
        title = ["NULL"]
    if not date:
        date = ["NULL"]

    # Find span for broad text
    first_broad_span = collected[0][0][0]
    last_broad_span = collected[-1][0][1]
    
    print(first_broad_span, last_broad_span)

    # correct span for broad texts now when broadtext chunk span is found
    collected = [((n[0][0] - first_broad_span, n[0][1] - first_broad_span), n[1]) for n in collected]

    for match in re.finditer(PATTERN_LINK_TEXT, source_page[first_broad_span:last_broad_span]):
        print(match)
        text = match.group() + " "
        span = match.span()
        data = (span, text)
        collected.append(data)

    collected.sort(key=lambda s: s[0])
    text_final = ""
    for data in collected:
        text_final += data[1]

    if not title:
        title = ["NULL"]

    return title[0], text_final, date[0]

In [72]:
str(soup)[90870:91206]

'Ryske oppositionsledaren Aleksej Navalnyj – som hålls fängslad – tvingas titta på statlig tv och propagandafilmer i mer än åtta timmar om dagen.</element><br/><element id="4SYwOUySBhhq5RcQ" type="preamble" xmlns="http://www.infomaker.se/idf/1.0">Det säger Navalnyj i sin första intervju med en nyhetsredaktion sedan gripandet i januari.'

In [71]:
collect_data(str(soup))

90870 91206


('Navalnyj: Tvingas titta på tv i fängelset',
 'Ryske oppositionsledaren Aleksej Navalnyj – som hålls fängslad – tvingas titta på statlig tv och propagandafilmer i mer än åtta timmar om dagen. Det säger Navalnyj i sin första intervju med en nyhetsredaktion sedan gripandet i januari. ',
 '"2021-08-25T19:01:48Z"')